In [1]:
import glob
import time
import numpy             as np
import pandas            as pd
import tables            as tb
import matplotlib.pyplot as plt

from invisible_cities.io.pmaps_io import load_pmaps_as_df

to_df = pd.DataFrame.from_records

In [2]:
dtype = "pmaps"
run_number = 7255

icversion_datadir  = f"/Users/gonzalo/Documents/NEXT/ireneProblem/DATA/{run_number}/v0.9.9/"
testprod_datadir   = f"/Users/gonzalo/Documents/NEXT/ireneProblem/DATA/{run_number}/IC-testprod/"
testprod_p_datadir = f"/Users/gonzalo/Documents/NEXT/ireneProblem/DATA/{run_number}/IC-testprod-p/"

icversion_files  = glob.glob(icversion_datadir  + f"*")
testprod_files   = glob.glob(testprod_datadir   + f"*")
testprod_p_files = glob.glob(testprod_p_datadir + f"*")

icversion_files.sort()
testprod_files.sort()
testprod_p_files.sort()

In [3]:
def loop_in_events(Spmap, SPMTpmap, EVENTS):
    n, time, energy, height, width, rms, most_active_pmt, mact_percentage = [], [], [], [], [], [], [], []
    events = []

    for ev in EVENTS:
        S    = Spmap   [Spmap   ["event"]==ev]
        SPMT = SPMTpmap[SPMTpmap["event"]==ev]

        PEAKS = np.unique(S["peak"])
        if len(PEAKS)>0: n.append(np.max(PEAKS)+1)
        else:            n.append(0)

        for pk in PEAKS:
            #Total
            times = S[S["peak"]==pk]["time"].values
            enes  = S[S["peak"]==pk]["ene"] .values

            events.append(ev)
            time  .append(times[np.argmax(enes)]/10**3)
            height.append(np.max(enes))
            energy.append(np.sum(enes))
            width .append(times[-1]-times[0])

            PMTS  = np.unique(SPMT["npmt"])
            esum = []
            for pmt in PMTS:
                enes = SPMT[SPMT["npmt"]==pmt]["ene"].values
                esum.append( np.sum(enes[enes>0]) )
            most_active_pmt.append(PMTS[np.argmax(esum)])
            mact_percentage.append( np.max(esum)/np.sum(esum)*100)
    return events, n, time, height, energy, width, most_active_pmt , mact_percentage

In [4]:
def Pmaps_PMTSignal_info(pmapdir, npmaps, outfilename):
    n_events_in_pmap = []

    nS1s, time_S1, height_S1, energy_S1, width_S1, rms_S1, most_active_pmt_S1 = [], [], [], [], [], [], []
    nS2s, time_S2, height_S2, energy_S2, width_S2, rms_S2, most_active_pmt_S2 = [], [], [], [], [], [], []
    mact_percentage_S1, mact_percentage_S2 = [], []

    events_S1, events_S2 = [], []
    evts = []
    file_number = []

    FILES = glob.glob(pmapdir + "/*")
    FILES.sort()
    FILES = FILES[:npmaps]

    for pmap in FILES:

        if pmap.split('_')[-3]=="v0.9.9":
            file_number.append(int(pmap.split('/')[-1].split('_')[1]))
        else:
            file_number.append(int(pmap.split('_')[-1][:4]))


        S1pmap, S2pmap, _, S1PMTpmap, S2PMTpmap = load_pmaps_as_df(pmap)

        EVENTS = np.unique(S2pmap["event"].values)
        evts.extend(EVENTS)
        n_events_in_pmap.append( len(EVENTS) )

        ######### S1 DATA ###########
        events, n, time, height, energy, width, most_active_pmt, mact_per = loop_in_events(S1pmap, S1PMTpmap, EVENTS)

        nS1s.extend(n)
        events_S1.extend(events)
        time_S1.extend(time)
        height_S1.extend(height)
        energy_S1.extend(energy)
        width_S1.extend(width)
        most_active_pmt_S1.extend(most_active_pmt)
        mact_percentage_S1.extend(mact_per)

        ######### S2 DATA ############
        events, n, time, height, energy, width, most_active_pmt, mact_per = loop_in_events(S2pmap, S2PMTpmap, EVENTS)

        nS2s.extend(n)
        events_S2.extend(events)
        time_S2.extend(time)
        height_S2.extend(height)
        energy_S2.extend(energy)
        width_S2.extend(width)
        most_active_pmt_S2.extend(most_active_pmt)
        mact_percentage_S2.extend(mact_per)


    S1_df = pd.DataFrame({
                'event':  events_S1,
                'time':   time_S1,
                'height': height_S1,
                'energy': energy_S1,
                'width':  width_S1,
                'mactPMT': most_active_pmt_S1,
                'mact_perc': mact_percentage_S1})


    S2_df = pd.DataFrame({
                'event':  events_S2,
                'time':   time_S2,
                'height': height_S2,
                'energy': energy_S2,
                'width':  width_S2,
                'mactPMT':most_active_pmt_S2,
                'mact_perc': mact_percentage_S2})

    nS_by_event = pd.DataFrame({
                'event': evts ,
                'nS1s':   nS1s,
                'nS2s':   nS2s})

    nevent_by_pmap = pd.DataFrame({
                'file': file_number ,
                'nevent':   n_events_in_pmap})


    S1_df         .to_hdf(outfilename, key='S1')
    S2_df         .to_hdf(outfilename, key='S2')
    nS_by_event   .to_hdf(outfilename, key='nS')
    nevent_by_pmap.to_hdf(outfilename, key='nevent')

In [37]:
t0 = time.perf_counter()
pmapdir = icversion_datadir
npmaps = 10
outfilename = "/Users/gonzalo/Documents/NEXT/ireneProblem/DATA/summary_pmtsignal_icversion.h5"

Pmaps_PMTSignal_info(pmapdir, npmaps, outfilename)
time.perf_counter() - t0

20.263000775000023

In [6]:
pd.read_hdf(outfilename, key="nS").head()

,event,nS1s,nS2s
0,7,4,1
1,13,3,1
2,15,2,1
3,17,14,1
4,23,18,1


In [7]:
pd.read_hdf(outfilename, key="S1").head()

,event,time,height,energy,width,mactPMT,mact_perc
0,7,277.200,0.745038,0.919726,150.0,5,15.460323
1,7,306.325,14.684610,80.435486,300.0,5,15.460323
2,7,410.225,1.108001,2.015945,325.0,5,15.460323
3,7,743.200,0.751289,1.516431,225.0,5,15.460323
4,13,340.500,0.601194,0.701673,200.0,0,10.272127


In [8]:
pd.read_hdf(outfilename, key="S2").head()

,event,time,height,energy,width,mactPMT,mact_perc
0,7,806.496437,8007.359863,58357.921875,21756.9375,1,11.740284
1,13,805.483313,10046.357422,199127.890625,67425.1250,1,11.733124
2,15,805.474938,10167.400391,56339.335938,14556.6875,1,11.586146
3,17,805.485812,11752.015625,101576.281250,15666.8125,1,11.678343
4,23,803.511125,11541.181641,50092.656250,10787.6250,1,11.410416


In [9]:
pd.read_hdf(outfilename, key="nevent").head()

,file,nevent
0,0,38
1,1,43
